In [2]:
import cv2
import os
import argparse
import matplotlib.pyplot as plt
import os
from time import time
from sklearn import metrics
import Dataset as dataset
import bag_of_visual_words as bovw
import mpeg7_color_layout as mpeg7
import candidate_selection as cand_selec
import constraint_creation as gen_cons
import sift as SIFT
import PC_Kmeans as PCK
import COP_Kmeans as COPK
# --------Acepting Input----------
# Initialize parser
parser = argparse.ArgumentParser()
# Adding arguments
parser.add_argument("--k", required=True)
parser.add_argument("--query", required=True)
# ---------------------------------

# -------------- Parameter Declaration -------------------
img_folder_path = '\\VOCtrainval_06-Nov-2007\\VOCdevkit\\VOC2007\\JPEGImages\\'
img_class_path = '\\VOCtrainval_06-Nov-2007\\VOCdevkit\\VOC2007\\ImageSets\\Main\\processed_files\\'
img_class_set_names = [{'name': 'aeroplane', 'type': 'vehicle'},
                       {'name': 'bicycle', 'type': 'vehicle'},
                       {'name': 'bird', 'type': 'animal'},
                       {'name': 'boat', 'type': 'vehicle'},
                       {'name': 'bottle', 'type': 'furniture'},
                       {'name': 'bus', 'type': 'vehicle'},
                       {'name': 'car', 'type': 'vehicle'},
                       {'name': 'cat', 'type': 'animal'},
                       {'name': 'chair', 'type': 'furniture'},
                       {'name': 'cow', 'type': 'animal'},
                       {'name': 'diningtable', 'type': 'furniture'},
                       {'name': 'dog', 'type': 'animal'},
                       {'name': 'horse', 'type': 'animal'},
                       {'name': 'motorbike', 'type': 'vehicle'},
                       {'name': 'person', 'type': 'person'},
                       {'name': 'pottedplant', 'type': 'furniture'},
                       {'name': 'sheep', 'type': 'animal'},
                       {'name': 'sofa', 'type': 'furniture'},
                       {'name': 'train', 'type': 'vehicle'},
                       {'name': 'tvmonitor', 'type': 'electronics'}]
# BOVW parameters
bovw_kmeans_model = None
tam_patch = (15, 15)
n_patches = 250
random_state = 1
n_dic = 50  # size of the dictionary

In [3]:
def gen_clus(c_name, cand_img, cons, f_name):
    '''Generate and Print Constrainted Cluster\n
       Parameters
           c_name: cluster name
           cand_img: candidate image list
           cons: Constraint class object
           f_name: feature name
    '''
    if c_name == "COP":
        '''---------------- COP-Kmeans ---------------'''
        c_kmeans = COPK.COP_KMeans(
            len(cons.descripList), cons.ml_g, cons.cl_g, f_name)
        c_kmeans.fit(cons.x)
    else:
        '''---------------- PC-Kmeans ---------------'''
        c_kmeans = PCK.PC_Kmeans(len(cons.descripList), cons.ml_g, cons.cl_g,
                              cons.neighborhoods, cons.y)
        c_kmeans.fit(cons.x)

    print(f'{c_name} Kmeans Clusters:')
    for i in range(len(c_kmeans.clusters)):
        print(f'Cluster {i+1} :')
        count = 0
        for index in c_kmeans.clusters[i]:
            count += 1
            if count != len(c_kmeans.clusters[i]):
                print(f'{cand_img[index].filename}, ', end='')
            else:
                print(f'{cand_img[index].filename}.')

    return c_kmeans, 

def execute(f_name, query_img, k, img_classSet, img_dataset, n_imgs):
    '''Generate and Print Constrainted Cluster\n
       Parameters
           f_name: cluster name
           query_img: the query image
           k: k neighbours for kNN
           img_classSet: label class array
           img_dataset: image dataset
           n_imgs: Length of img List
    '''
    if f_name == "BOVW":
        print('----------- Bag of Visual Words -----------')
        t0 = time()
        bovw_param = bovw.BOVW(random_state=random_state,
                               n_patches=n_patches, tam_patch=tam_patch, n_dic=n_dic)
        features = bovw.execute_Bovw(
            img_dataset, bovw_param, n_imgs)
        query_feature = bovw.get_bovw_features(query_img, bovw_param)
        print("BOVW features Creation time: %0.3fs" % (time() - t0))
    elif f_name == "SIFT":
        print('----------- SIFT -----------')
        t0 = time()
        sift_keypoints, features = SIFT.execute_sift(img_dataset, 1000)
        query_feature = SIFT.get_sift_features(query_img)
        print("SIFT features Creation time: %0.3fs" % (time() - t0))
    elif f_name == "MPEG7":
        print('----------- MPEG7 -----------')
        t0 = time()
        features = mpeg7.execute_mpeg7(img_dataset)
        query_feature = mpeg7.get_mpeg7_features(query_img)
        print("MPEG7 features Creation time: %0.3fs" % (time() - t0))

    '''----------- Candidate Selection ---------------'''
    t0 = time()
    cand_img, cand_features = cand_selec.select_candidates(
        f_name, k, features, query_feature, img_dataset, n_imgs)
    print("Candidate Selection time: %0.3fs" % (time() - t0))

    '''----------- Constraint Creation ---------------'''
    t0 = time()
    cand_img, cons = gen_cons.generate_constraints(
        cand_img, cand_features, img_classSet)
    print("Constraint Creation time: %0.3fs" % (time() - t0))

    return cand_img, cons

In [4]:
t0 = time()
# Read arguments from command line
args = parser.parse_args(["--k", "50", "--query", "000036"])
k = int(args.k)
query = args.query
path_query = os.getcwd() + img_folder_path + query + '.jpg'
query_img = cv2.imread(path_query)

'''----------- Creating Data Set ------------------'''
image_classSet = dataset.extract_imageDescrip(
    img_class_set_names, img_class_path)
image_dataset, n_imgs = dataset.create_dataset(img_folder_path)
print("Data Set Creation time: %0.3fs" % (time() - t0))


Data Set Creation time: 7.876s


In [5]:
'''Extract image features, candidates using knn & create constraints for given feature name'''
cand_img_mpeg7, cons_mpeg7 = execute("MPEG7", query_img, k,
                                     image_classSet, image_dataset, n_imgs)
cand_img_bovw, cons_bovw = execute("BOVW", query_img, k,
                                   image_classSet, image_dataset, n_imgs)


----------- MPEG7 -----------
MPEG7 features Creation time: 14.840s
Candidate Selection time: 0.095s
Constraint Creation time: 0.051s
----------- Bag of Visual Words -----------
------------------ Feature Extraction - Bag of Visual Words - START ----------
Patches extracted to create dictionary of features
Total of images =  5011
Size of each array of patches =  (250, 675)
Patches Creation time: 36.080s
Instances =  1252750  size =  10
Created LBP feature spaces
	patches =  1252750  size =  10
LBP Instance Creation time: 53.246s
Kmeans fitting time: 247.862s
Creation of image features: 11.582s
Number of images and features =  (5011, 50)
------------------ Feature Extraction - Bag of Visual Words - END ------------
BOVW features Creation time: 351.224s
Candidate Selection time: 0.056s
Constraint Creation time: 0.076s


In [13]:
'''Extract image features, candidates using knn & create constraints for given feature name'''
cand_img_sift, cons_sift = execute("SIFT", query_img, k,
                         image_classSet, image_dataset, n_imgs)


----------- SIFT -----------


KeyboardInterrupt: 

In [12]:
'''Generate Clusters'''
mpeg7_copk_clus = gen_clus("COP", cand_img_mpeg7, cons_mpeg7, "MPEG7")
mpeg7_pck_clus = gen_clus("PC", cand_img_mpeg7, cons_mpeg7, "MPEG7")


COP Kmeans Clusters:
Cluster 1 :
009524, 001405, 000125, 005315, 002669, 000382, 007122, 000862, 008892, 001686, 008944.
Cluster 2 :
000036, 002938, 009700, 009144, 006751, 001203, 005854, 008258, 005805, 005202, 000849, 001348, 002969, 006026, 004537, 006091, 004368, 003772, 001209.
Cluster 3 :
002287, 003189, 003044, 000729, 008989, 000460.
Cluster 4 :
008008, 007855, 001427, 006162, 006585, 009717, 004391, 000380, 003657, 002241, 008968, 005897, 006258, 003567.
PC Kmeans Clusters:
Cluster 1 :
000036, 002938, 009700, 009144, 006751, 001203, 005854, 008258, 005805, 005202, 000849, 001348, 002969, 006026, 004537, 006091, 004368, 003772, 001209.
Cluster 2 :
008008, 007855, 001427, 006162, 006585, 009717, 004391, 000380, 003657, 002241, 008968, 005897, 006258, 003567.
Cluster 3 :
009524, 001405, 000125, 005315, 002669, 000382, 007122, 000862, 008892, 001686, 008944.
Cluster 4 :
002287, 003189, 003044, 000729, 008989, 000460.


In [8]:
'''Generate Clusters'''
bovw_copk_clus = gen_clus("COP", cand_img_bovw, cons_bovw, "BOVW")
bovw_pck_clus = gen_clus("PC", cand_img_bovw, cons_bovw, "BOVW")


COP Kmeans Clusters:
Cluster 1 :
002171, 007683, 004008, 009789, 009500, 005064.
Cluster 2 :
001878, 003845, 001158, 004682, 008835, 004333, 000868, 004140.
Cluster 3 :
000552, 005893, 005639, 006719, 006371, 006736, 000871.
Cluster 4 :
000036, 006234, 002199, 005006, 000225, 004264, 005352, 005812, 006495, 008773, 005169, 002804, 008115, 009323, 008815, 000682, 004634, 002977, 003645, 007230, 000359, 007713, 002330, 004760, 005161, 003772, 000416, 009429, 001192.
PC Kmeans Clusters:
Cluster 1 :
000036, 006234, 002199, 005006, 000225, 004264, 005352, 005812, 006495, 008773, 005169, 002804, 008115, 009323, 008815, 000682, 004634, 002977, 003645, 007230, 000359, 007713, 002330, 004760, 005161, 003772, 000416, 009429, 001192.
Cluster 2 :
002171, 007683, 004008, 009789, 009500, 005064.
Cluster 3 :
000552, 005893, 005639, 006719, 006371, 006736, 000871.
Cluster 4 :
001878, 003845, 001158, 004682, 008835, 004333, 000868, 004140.


In [ ]:
'''Generate Clusters'''
sift_copk_clus = gen_clus("COP", cand_img_sift, cons_sift, "SIFT")
sift_pck_clus = gen_clus("PC", cand_img_sift, cons_sift, "SIFT")
